# Prompt chaining and sequencing

This notebook explores the concepts of prompt chaining and sequencing in the context of working with LLMs. As AI applications become more sophisticated, the ability to handle more complex, multi-step tasks becomes increasingly critical. In many scenarios, a single prompt or model response isn't sufficient to solve complex problems or generate sophisticated outputs. Instead, tasks often need to be broken down into smaller, more manageable parts, each requiring a specific response or analysis.

Prompt chaining and sequencing are techniques designed to address this challenge. These approaches enable us to guide LLMs through a series of interconnected steps, where the output from one prompt becomes the input for the next. This structured flow is akin to a step-by-step process, where each prompt builds on the last, ensuring that the model's outputs remain coherent and aligned with the overall goal.

This method is particularly valuable for tasks that require nuanced reasoning, multiple stages of processing, or decisions that need to be informed by prior steps. By using these techniques, we can construct more sophisticated AI-driven applications that are capable of handling complex problems, ensuring that the process is not only systematic but also more transparent and interpretable.

In [1]:
import os
import re

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

## Basic Prompt Chaining
In the first example, we will demonstrate simple prompt chaining. This involves creating two simple prompts: one to generate a short story based on a genre, and another to summarize that story. The output of the first prompt (the story) will be passed as input to the second prompt (the summary).

In [3]:
# Define prompt templates
story_prompt = PromptTemplate(
    input_variables=["genre"],  # Define 'genre' as an input variable for the prompt
    template="Write a short {genre} story in 3-4 sentences."  # Template for generating the story
)

summary_prompt = PromptTemplate(
    input_variables=["story"],  # Define 'story' as an input variable for the prompt
    template="Summarize the following story in one sentence:\n{story}"  # Template for summarizing the story
)

# Chain the prompts
def story_chain(genre):
    """Generate a story and its summary based on a given genre.

    Args:
        genre (str): The genre of the story to generate.

    Returns:
        tuple: A tuple containing the generated story and its summary.
    """
    # Generate the story based on the input genre using the first prompt
    story = (story_prompt | llm).invoke({"genre": genre}).content

    # Generate the summary based on the story generated in the previous step
    summary = (summary_prompt | llm).invoke({"story": story}).content

    return story, summary  # Return both the story and its summary

# Test the chain
genre = "science fiction"  # Define the genre for the story
story, summary = story_chain(genre)  # Run the prompt chain
print(f"Story: {story}\n\nSummary: {summary}")

Story: In the year 2147, humanity discovered a hidden planet orbiting a dying sun, inhabited by sentient beings of pure light. These Lumarians shared a unique ability: they could merge their consciousness with any living creature, revealing the universe's secrets through shared memories. As an interstellar diplomat, Elara connected with a Lumarian named Zephyr, experiencing the birth of stars and the fall of civilizations. In that moment, she understood that knowledge was not just power, but a shared journey, and she vowed to protect the Lumarians from those who sought to exploit their gift.

Summary: In 2147, interstellar diplomat Elara forms a profound connection with a Lumarian named Zephyr, discovering the power of shared knowledge and vowing to protect their unique abilities from exploitation.


1. Prompt template definitions: The `PromptTemplate` class from LangChain is used to define templates for each prompt. These templates allow for dynamic insertion of variables into the prompt, enabling the model to generate content based on specific inputs.
   - `story_prompt`: This prompt is designed to generate a short story based on a given genre. The genre is provided as an input variable and is used within the template to generate a unique story based on that genre.
   - `summary_prompt`: This prompt takes a story and asks the model to summarize it in one sentence. The story is passed as the input variable, and the model is instructed to condense the content into a brief, one-sentence summary.
2. The `story_chain` function:
   - First step (story generation): The `story_prompt` is invoked with the provided `genre`, and the model generates a short story. This is done using the `invoke` method, where the genre is passed as input to the prompt. The `| llm` syntax is used to chain the prompt template to the language model. The model processes the prompt and returns a generated response, which is accessed via the `.content` attribute.
   - Second Step (summary generation): The `story` generated in the first step is passed into the `summary_prompt` as input, and the model then produces a one-sentence summary of that story.

By using prompt chaining in this manner, we can achieve more structured and logical AI outputs, where each step builds on the previous one, making the results more coherent and purposeful.

By starting with a simple task (story generation and summarization), we can expand this technique to more complex workflows, where multiple steps are required to solve a problem or generate sophisticated content.


## Sequential prompting
In this section, we will explore sequential prompting, a more complex approach where multiple prompts are used in a series to analyze a given text in stages. We will break down the analysis into three specific tasks: identifying the main theme, determining the tone, and extracting the key takeaways. Each step builds upon the previous one, ensuring that the output is well-structured and coherent.

By chaining these prompts, we guide the model through a multi-step process that enables deeper and more structured insights into the text.

In [4]:
# Define prompt templates for each step of the analysis
theme_prompt = PromptTemplate(
    input_variables=["text"],  # Takes the full text as input
    template="Identify the main theme of the following text:\n{text}"  # Instructions to identify the theme
)

tone_prompt = PromptTemplate(
    input_variables=["text"],  # Takes the full text as input
    template="Describe the overall tone of the following text:\n{text}"  # Instructions to identify the tone
)

takeaway_prompt = PromptTemplate(
    input_variables=["text", "theme", "tone"],  # Takes the text, theme, and tone as input
    template="Given the following text with the main theme '{theme}' and tone '{tone}', what are the key takeaways?\n{text}"  # Instructions to extract key takeaways
)

def analyze_text(text):
    """Perform a multi-step analysis of a given text.

    Args:
        text (str): The text to analyze.

    Returns:
        dict: A dictionary containing the theme, tone, and key takeaways of the text.
    """
    # Step 1: Identify the theme of the text
    theme = (theme_prompt | llm).invoke({"text": text}).content

    # Step 2: Identify the tone of the text
    tone = (tone_prompt | llm).invoke({"text": text}).content

    # Step 3: Extract the key takeaways based on the identified theme and tone
    takeaways = (takeaway_prompt | llm).invoke({"text": text, "theme": theme, "tone": tone}).content

    # Return a dictionary containing all three pieces of information
    return {"theme": theme, "tone": tone, "takeaways": takeaways}

# Test the sequential prompting
sample_text = "The rapid advancement of artificial intelligence has sparked both excitement and concern among experts. While AI promises to revolutionize industries and improve our daily lives, it also raises ethical questions about privacy, job displacement, and the potential for misuse. As we stand on the brink of this technological revolution, it's crucial that we approach AI development with caution and foresight, ensuring that its benefits are maximized while its risks are minimized."

analysis = analyze_text(sample_text)  # Perform the analysis on the sample text
# Print the analysis results
for key, value in analysis.items():
    print(f"{key.capitalize()}: {value}\n")

Theme: The main theme of the text is the dual nature of artificial intelligence advancements, highlighting both their potential benefits and associated ethical concerns. It emphasizes the need for careful and responsible development to maximize positive outcomes while mitigating risks.

Tone: The overall tone of the text is balanced and thoughtful. It expresses a mixture of excitement and concern regarding the advancements in artificial intelligence. The language conveys optimism about the potential benefits of AI, while also highlighting the need for caution and ethical considerations. This duality reflects a responsible and measured perspective on a rapidly evolving technology.

Takeaways: Here are the key takeaways from the provided text:

1. **Dual Nature of AI Advancements**: The text emphasizes that artificial intelligence has both significant potential benefits and serious ethical concerns.

2. **Potential Benefits**: AI is positioned as a transformative technology that can revo

1. Prompt templates:
   - `theme_prompt`: This template asks the model to identify the main theme of a given text. The input is the entire text, and the output should be a summary of the central idea or theme.
   - `tone_prompt`: This template requests the model to describe the tone of a text. The tone can be classified as positive, negative, neutral, formal, informal, etc.
   - `takeaway_prompt`: This is the final prompt, which uses theme and tone along with a text to extract the key takeaways or conclusions.
2. The `analyze_text` function:
   - Step 1 (theme identification): The function first invokes the `theme_prompt` to extract the main theme of the text.
   - Step 2 (tone analysis): Next, it invokes the `tone_prompt` to analyze the tone of the text.
   - Step 3 (key takeaways): Using the theme and tone identified in the previous steps, the function invokes the `takeaway_prompt` to extract the key takeaways from the text.

In this example, sequential prompting was used to break down a complex analysis task into a series of manageable steps. Each prompt builds on the information generated by the previous one, allowing for a detailed and structured analysis of the text. This technique is especially useful when dealing with tasks that require multiple stages of reasoning or decision-making.

## Dynamic prompt generation
In this section, we will build a dynamic question-answering system that generates follow-up questions based on previous answers. This approach allows the system to engage in a continuous dialogue where each new question depends on the previous response, making the interaction more natural and context-aware.
- The system begins with an initial question.
- For each subsequent step, it generates an answer to the current question, then uses that answer to generate a follow-up question.
- The follow-up question is dynamically created based on the answer to the previous question, allowing the conversation to evolve in a coherent way.

In [5]:
# Define prompt templates
# This template generates an answer to the question
answer_prompt = PromptTemplate(
    input_variables=["question"],  # Takes a question as input
    template="Answer the following question concisely:\n{question}"  # Instruction for generating a concise answer
)

# This template generates a follow-up question based on the previous question and its answer
follow_up_prompt = PromptTemplate(
    input_variables=["question", "answer"],  # Takes both the previous question and answer as input
    template="Based on the question '{question}' and the answer '{answer}', generate a relevant follow-up question."  # Instruction to generate a relevant follow-up question
)

def dynamic_qa(initial_question, num_follow_ups=3):
    """Conduct a dynamic Q&A session with follow-up questions.

    Args:
        initial_question (str): The initial question to start the Q&A session.
        num_follow_ups (int): The number of follow-up questions to generate.

    Returns:
        list: A list of dictionaries containing questions and answers.
    """
    qa_chain = []  # Initialize an empty list to store the question-answer pairs
    current_question = initial_question  # Set the initial question

    # Loop for generating the initial question and the required number of follow-up questions
    for _ in range(num_follow_ups + 1):  # +1 for the initial question
        # Get the answer to the current question
        answer = (answer_prompt | llm).invoke({"question": current_question}).content
        # Add the current question and its answer to the chain
        qa_chain.append({"question": current_question, "answer": answer})

        if _ < num_follow_ups:  # Generate follow-up for all but the last iteration
            # Generate the next question based on the current question and answer
            current_question = (follow_up_prompt | llm).invoke({"question": current_question, "answer": answer}).content

    return qa_chain  # Return the entire list of question-answer pairs

# Test the dynamic Q&A system with an initial question
initial_question = "What are the potential applications of quantum computing?"
qa_session = dynamic_qa(initial_question)  # Conduct the dynamic Q&A session

# Print the Q&A session
for i, qa in enumerate(qa_session):
    print(f"Q{i+1}: {qa['question']}")
    print(f"A{i+1}: {qa['answer']}\n")

Q1: What are the potential applications of quantum computing?
A1: Quantum computing has potential applications in various fields, including:

1. **Cryptography**: Enhancing security through quantum key distribution and breaking classical encryption methods.
2. **Drug discovery**: Simulating molecular interactions to accelerate the development of new pharmaceuticals.
3. **Optimization problems**: Solving complex optimization tasks in logistics, finance, and operations research more efficiently.
4. **Machine learning**: Improving data analysis and pattern recognition through quantum-enhanced algorithms.
5. **Material science**: Discovering new materials with desired properties by simulating atomic structures.
6. **Climate modeling**: Analyzing large datasets for more accurate climate predictions and simulations.
7. **Artificial intelligence**: Enhancing algorithms for faster training and improved decision-making.

These applications leverage quantum mechanics principles, such as superpos

1. Prompt templates:
   - `answer_prompt`: This template generates a concise answer to any question. It simply requires the input of a question, and the model responds with a relevant answer.
   - `follow_up_prompt`: This template generates a follow-up question based on the current question and its associated answer. It takes both the question and the answer as input and generates a follow-up question to continue the conversation.
2. The `dynamic_qa` function: The function takes an initial question to start the Q&A session, and an optional number of follow-up questions (default is 3).
     - The function starts with the initial question and generates an answer using the `answer_prompt` template.
     - It then stores the question-answer pair in a list (`qa_chain`).
     - For each subsequent iteration, the function uses the previous question and answer to generate a follow-up question using the `follow_up_prompt` template.
     - The process repeats until the desired number of follow-up questions has been generated.

The system is capable of maintaining a conversation by generating follow-up questions based on previous answers. The ability to dynamically generate questions makes the system more interactive and capable of engaging in deeper dialogues. The process uses prompt chaining, where the output of one step (the answer) is passed as input to the next (the follow-up question). This ensures that the generated content is contextually relevant.


## Error handling and validation
In this section, we focus on error handling and validation techniques to make our prompt chains more robust and reliable. When working with AI models, it is essential to ensure that the generated outputs are both correct and relevant. We will incorporate multiple steps to handle potential errors, validate the model’s responses, and retry the generation process if necessary.
- Error handling: Ensures that the system continues to function properly even if something goes wrong during the prompt generation process.
- Validation: Confirms that the generated output meets the required conditions (e.g., relevance to the topic).

The process involves generating a 4-digit number related to a specific topic, validating that the number is correct, and retrying if it is not.

In [6]:
# Define prompt templates
# Template to generate a 4-digit number related to a specific topic
generate_prompt = PromptTemplate(
    input_variables=["topic"],  # Takes the topic as input
    template="Generate a 4-digit number related to the topic: {topic}. Respond with ONLY the number, no additional text."
)

# Template to validate if the generated number is relevant to the topic
validate_prompt = PromptTemplate(
    input_variables=["number", "topic"],  # Takes the generated number and topic as input
    template="Is the number {number} truly related to the topic '{topic}'? Answer with 'Yes' or 'No' and explain why."
)

# Function to extract a 4-digit number from a string
def extract_number(text):
    """Extract a 4-digit number from the given text.

    Args:
        text (str): The text to extract the number from.

    Returns:
        str or None: The extracted 4-digit number, or None if no valid number is found.
    """
    match = re.search(r'\b\d{4}\b', text)  # Search for a 4-digit number in the text
    return match.group() if match else None  # Return the number if found, else return None

# Function to generate a number and validate its relevance
def robust_number_generation(topic, max_attempts=3):
    """Generate a topic-related number with validation and error handling.

    Args:
        topic (str): The topic to generate a number for.
        max_attempts (int): Maximum number of generation attempts.

    Returns:
        str: A validated 4-digit number or an error message.
    """
    for attempt in range(max_attempts):  # Retry up to max_attempts if validation fails
        try:
            # Generate a 4-digit number related to the topic
            response = (generate_prompt | llm).invoke({"topic": topic}).content
            # Extract the 4-digit number from the response
            number = extract_number(response)

            # If no valid number is found, raise an exception
            if not number:
                raise ValueError(f"Failed to extract a 4-digit number from the response: {response}")

            # Validate the relevance of the number
            validation = (validate_prompt | llm).invoke({"number": number, "topic": topic}).content
            if validation.lower().startswith("yes"):  # Check if the validation was successful
                return number
            else:
                print(f"Attempt {attempt + 1}: Number {number} was not validated. Reason: {validation}")
        except Exception as e:
            # Print error message if an exception occurs
            print(f"Attempt {attempt + 1} failed: {str(e)}")

    return "Failed to generate a valid number after multiple attempts."  # Return failure message if all attempts fail

# Test the robust number generation
topic = "World War II"
result = robust_number_generation(topic)
# Print the final result (either a valid number or an error message)
print(f"Final result for topic '{topic}': {result}")

Final result for topic 'World War II': 1945


1. Prompt templates:
   - `generate_prompt`: This template prompts the model to generate a 4-digit number that is related to a given topic. The model is instructed to provide only the number, with no additional text.
   - `validate_prompt`: This template asks the model to validate whether a number is truly related to the given topic. The model responds with either "Yes" or "No," along with an explanation.
2. `extract_number` function: This function uses a regular expression (`re.search`) to extract a 4-digit number from a string. If no number is found, it returns `None`. The function searches for a 4-digit number using the regular expression pattern `\b\d{4}\b`, which matches exactly four digits surrounded by word boundaries.
3. `robust_number_generation` function: Takes the topic for which a 4-digit number is to be generated, and the optional max_attempts parameter that specifies how many times the process will retry in case of failure.
    1. The function attempts to generate the number for the given topic by invoking the `generate_prompt` template. The response is processed to extract the number.
    2. If the number is invalid (i.e., not found or incorrect), an exception is raised.
    3. If a valid number is generated, it is validated by invoking the `validate_prompt` template to check if the number is actually related to the topic.
    4. If validation fails, the process retries up to the maximum number of attempts.
    5. If a valid number is generated and validated, it is returned. Otherwise, after the maximum number of attempts, the function returns a failure message.


The error handling and validation mechanism ensures that the system can gracefully handle failures and retry operations if necessary. By incorporating validation at each step and handling errors effectively, the system becomes more reliable and robust. This approach is essential when building AI applications that interact with unpredictable models, ensuring that the generated outputs are both correct and useful.
- Error handling: The function is wrapped in a `try-except` block to catch any potential errors (e.g., network errors, invalid number extraction). If an error occurs during number extraction or validation, it retries the process for up to `max_attempts`.
- Validation: After generating a number, the system validates its relevance to the given topic using a follow-up question (`validate_prompt`). If the validation fails, the process retries the generation and validation steps.